<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse LDA
    - Entrainement LDA
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 18/07/2020 

In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')

In [2]:
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 

C:\Users\naru_\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
tagText = TagText()
print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)

tagText.nombre_post_entree = 5000
tagText.precision = 5000

data_question = tagText.read_source()
data_question.head()


Data/
QuestionVsTags.csv


,body,title,tags,SelectedTags,Name
0,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo><pre-c...,"git,version-control,",Question
2,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><version-control><git-branch><git-push><g...,"git,version-control,",Question
3,<p>What are the differences between <code>git ...,What is the difference between 'git pull' and ...,<git><version-control><git-pull><git-fetch>,"git,version-control,",Question
4,<p>What is the use of the <code>yield</code> k...,"What does the ""yield"" keyword do?",<python><iterator><generator><yield><coroutine>,"python,iterator,",Question


In [4]:
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
print(data_preprocessed[1:9])

1    accidentally committed wrong files git pushed ...
2    want delete branch locally remotely failed att...
3                       differences git pull git fetch
4    use yield keyword python do example im trying ...
5    ive messing around json time pushing text hurt...
6    mistakenly added files git using command git a...
7    reading hidden features dark corners cstl comp...
8    want rename remote branch described rename mas...
Name: body, dtype: object


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
III.2 - Train/Test</p>

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Sampling dataset
vectorizer_X = TfidfVectorizer(
            max_df=0.95,
            min_df=2,
            max_features=50000,
            stop_words='english'
                              )

#Y
y_all = [
    item[:-1].split(',')#-1 car il y a un ',' à la fin de la ligne
    for item in data_question['SelectedTags']
]

#print(y_train_tag)
lb = tagText.MultiLabelBinarizer()
Y_all = lb.fit_transform(y_all)


# 80/20 split
X_lda_train, X_lda_test, y_lda_train, y_lda_test = train_test_split(
    data_preprocessed, y_all, test_size=0.2,train_size=0.8, random_state=0)
y_lda_train = lb.transform(y_lda_train)
y_lda_test = lb.transform(y_lda_test)

# TF-IDF matrices
X_tfidf_train = vectorizer_X.fit_transform(X_lda_train)
X_tfidf_test = vectorizer_X.transform(X_lda_test)

In [6]:
print(y_lda_test)
print(Y_all)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [7]:
print(len(y_lda_train))

4000


In [8]:
type(X_tfidf_train)
print(X_tfidf_train)

  (0, 4530)	0.2770785051326141
  (0, 5696)	0.2252549744174165
  (0, 2551)	0.28640531275975895
  (0, 2298)	0.14596492083710486
  (0, 2864)	0.2646929887216864
  (0, 2311)	0.17579442970613238
  (0, 1767)	0.20773835932418794
  (0, 936)	0.6100653510305448
  (0, 3426)	0.5028448259103764
  (1, 3265)	0.02131652323813375
  (1, 6080)	0.07800428356342129
  (1, 767)	0.08491887052981097
  (1, 4444)	0.03588998780584211
  (1, 435)	0.07364640262074827
  (1, 1246)	0.15600856712684258
  (1, 6324)	0.0495994696971064
  (1, 1464)	0.1540675075500231
  (1, 5668)	0.04557841083226449
  (1, 638)	0.10486630355579327
  (1, 1159)	0.07795261673002354
  (1, 5050)	0.09115682166452899
  (1, 984)	0.08885571320645763
  (1, 4683)	0.03536427238285232
  (1, 4479)	0.13311773629889018
  (1, 2672)	0.04819703840893963
  :	:
  (3998, 3634)	0.31200098064559983
  (3998, 1623)	0.4376610197110614
  (3998, 6238)	0.3710768270666014
  (3998, 1047)	0.3660042416592107
  (3998, 5029)	0.4870079990462219
  (3998, 690)	0.44974916594508857
 

In [9]:
idf = vectorizer_X.idf_
print(idf)

[7.6848617  7.50254014 7.90800525 ... 7.90800525 7.90800525 8.19568732]


In [10]:
dd=dict(zip(vectorizer_X.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])

like abcde


In [11]:
'''
print(dd['like'])
print(dd['consolewritelinesecond'])  # like is most common and forecast is least common among the news headlines.
'''

"\nprint(dd['like'])\nprint(dd['consolewritelinesecond'])  # like is most common and forecast is least common among the news headlines.\n"

In [12]:
print(dd)

{'texta': 8.195687320189126, 'logical': 7.097075031521015, 'self': 6.349860629690794, 'plugins': 6.521710886617454, 'indexed': 7.684861696423135, 'declaration': 6.158805392928086, 'refers': 7.214858067177399, 'welcome': 7.214858067177399, 'cow': 8.195687320189126, 'buffering': 7.908005247737345, 'oncreatebundle': 6.809392959069235, 'vertically': 6.461086264801019, 'imagepng': 8.195687320189126, 'colors': 6.729350251395698, 'codebase': 6.9917145158631895, 'concatenating': 7.908005247737345, 'deep': 7.214858067177399, 'yn': 8.195687320189126, 'panels': 8.195687320189126, 'supported': 6.075423783989034, 'effective': 7.097075031521015, 'calculating': 7.50254013962918, 'drive': 6.896404336058865, 'const': 5.710780670401125, 'gulp': 7.684861696423135, 'restarted': 7.214858067177399, 'class': 3.6313391287212893, '37': 7.50254013962918, 'information': 4.805663239125096, 'sizeof': 8.195687320189126, 'technically': 7.097075031521015, 'classes': 5.074791903681128, 'dropped': 7.50254013962918, 'vi

In [13]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(X_tfidf_train)


In [14]:
# composition of doc 0 for eg
print("Document 0: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i,": ",topic*100,"%")

Document 0: 
Topic  0 :  2.705746531705599 %
Topic  1 :  75.64767704057599 %
Topic  2 :  2.706327576510599 %
Topic  3 :  2.7057475315697723 %
Topic  4 :  2.705746119535403 %
Topic  5 :  2.705746382851403 %
Topic  6 :  2.705771999240231 %
Topic  7 :  2.7057458921623474 %
Topic  8 :  2.705745438222897 %
Topic  9 :  2.70574548762577 %


In [15]:
type(lda_top)
print(lda_top)

[[0.02705747 0.75647677 0.02706328 ... 0.02705746 0.02705745 0.02705745]
 [0.01007991 0.04152212 0.73412887 ... 0.01007973 0.01007974 0.01007973]
 [0.02125413 0.74674822 0.08320946 ... 0.02125425 0.02125416 0.02125422]
 ...
 [0.01779708 0.01779917 0.83982271 ... 0.01779709 0.01779708 0.01779784]
 [0.02920988 0.02921177 0.73710875 ... 0.02920987 0.02920988 0.02920988]
 [0.01978348 0.01978445 0.82194518 ... 0.01978349 0.01978348 0.0197858 ]]


In [16]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(
        n_estimators=100,
        max_depth=2,
        random_state=0
)
#lda_top=lda_model.fit_transform(X_tfidf_train)
clf.fit(lda_top, y_lda_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [17]:
classes = lb.classes_

no_top_words = 5
text = "git is the best source code"
text = tagText.preprocessing(text)
text = [text]
mytext = vectorizer_X.transform(text)
text_projection = lda_model.transform(mytext)
'''
print (text_projection)
print (text_projection.shape)
'''
predicted = clf.predict_proba(text_projection)
'''
print (predicted)
print (len(predicted))
'''
tempTag = [(1-item[0][0]) for item in predicted]
list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
'''
tempTag = predicted[0]
list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
'''
'''
print (predicted)
print (predicted.shape)
'''
list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
#print (list_id_sorted)

list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
#print(list_id_sorted_suggested)

print (str([classes[id] for id in list_id_sorted_suggested]))

['networking', 'ssh', 'cocoa-touch', 'asp.net', 'datetime']


In [18]:
def scoring(x_test, y_true, clf, lb, mode_supervise_with_lda = False, lda_model = None):
    nb_tag_1 = 0.0
    nb_tag_5 = 0.0
    classes = lb.classes_
    print(x_test.shape[0])
    no_top_words = 5
    for i in range(x_test.shape[0]):
        #for i in range(x_test.shape[0]):
        text_projection = x_test
        if(mode_supervise_with_lda):
            text_projection = lda_model.transform(x_test[i])
        '''
        print (text_projection)
        print (text_projection.shape)
        '''
        predicted = clf.predict_proba(text_projection)
        '''
        print (predicted)
        print (len(predicted))
        '''
        tempTag = [(1-item[0][0]) for item in predicted]
        list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
        '''
        tempTag = predicted[0]
        list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
        '''
        '''
        print (predicted)
        print (predicted.shape)
        '''
        list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
        #print (list_id_sorted)

        list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
        #print(list_id_sorted_suggested)
        prediction = [classes[id] for id in list_id_sorted_suggested]
        #print (str(prediction))
        
        
        l_y = [[i, x] for i, x in enumerate(y_true[i]) if x > 0]
        l_y_tagged = [x[0] for i, x in enumerate(l_y[:-no_top_words - 1:-1])]
        l_y_tags = [classes[id] for id in l_y_tagged]
        #print (l_y_tags)
        
        check_1 = False
        check_1 = any(item in prediction for item in l_y_tags)

        if check_1 is True:
            nb_tag_1 = nb_tag_1 + 1
        """    print("The list {} contains some elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """

        check_5 = False
        check_5 = all(item in prediction for item in l_y_tags)
        if check_5 is True:
            nb_tag_5 = nb_tag_5 + 1
        """    print("The list {} contains all elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        #str([tag for tag in y_true[i]]if tag ==1)
    return nb_tag_1, (100.0 * nb_tag_1 / float(x_test.shape[0])), nb_tag_5, (100.0 * nb_tag_5 / float(x_test.shape[0]))

scoring(X_tfidf_test, y_lda_test, clf, lb, True, lda_model)

1000


(29.0, 2.9, 2.0, 0.2)

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV.2-a - Data - Tag generator - Unsupervised - LDA</p>

In [19]:
def lda_prepare_tag(tagText, data_preprocessed, data_tag, no_tropics=32):
        '''
        prepare lda, topic ad tf vectorizer from data preprocessed
        '''
        from sklearn.ensemble import RandomForestClassifier
        
        tagText.n_topic = no_tropics
        documents = data_preprocessed[0:tagText.precision]#.unique()
        lda_tf, lda_tf_vectorizer = tagText.lda_init(documents)
        
        lda, score, perplexity = tagText.lda_train(lda_tf, no_tropics)
        
        print("Log Likelihood: ", score)
        print("Perplexity: ", perplexity)

        # See model parameters
        print(lda.get_params())

        lda_topicnames = ["Topic" + str(i) for i in range(lda.n_topics)]
        # Topic-Keyword Matrix
        lda_df_topic_keywords = tagText.pd.DataFrame(lda.components_)
        
        # Assign Column and Index
        lda_df_topic_keywords.columns = lda_tf_vectorizer.get_feature_names()
        lda_df_topic_keywords.index = lda_topicnames
        
        #tag
        topic_word = lda.topic_word_
        print("type(topic_word): {}".format(type(topic_word)))
        print("shape: {}".format(topic_word.shape))
        
        clf = RandomForestClassifier(
                n_estimators=100,
                max_depth=2,
                random_state=0
        )
        clf.fit(lda, y_lda_train)
        
        return lda, lda_df_topic_keywords, lda_tf_vectorizer, clf

In [20]:
lda_tagText, df_topic_keywords, tf_vectorizer, clf = lda_prepare_tag(tagText, X_tfidf_train, y_lda_train, 32)


AttributeError: lower not found

In [ ]:
def average_jaccard(y_true,y_pred):

    ''' It calculates Jaccard similarity coefficient score for each instance,and
    it finds their averange in percentage
    Parameters:
    y_true: truth labels
    y_pred: predicted labels
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100

In [ ]:
# most important words for each topic
vocab = vectorizer_X.get_feature_names()

for i, comp in enumerate(lda_tagText.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")